In [4]:
import sys; sys.path.append('../../')
import numpy as np
import pandas as pd
from DataPreparation.DataPreparation import read_data
from sklearn.metrics import classification_report
from mlpath import mlquest as mlq
from utils import load_hyperparameters, save_model, get_metrics, save_hyperparameters
from ModelAnalysis import cross_validation
from xgboost import XGBClassifier
model_name= 'XGBoost'

mlq.start_quest(model_name, table_dest="../../", log_defs=True)

/opt/anaconda3/envs/arm/lib/python3.9/site-packages/mlpath/mlquest/mlquest.py:66: UserWarning: Attempting to start a run while another one is active may cause data overwrite
  if mlquest.active == True: warnings.warn("Attempting to start a run while another one is active may cause data overwrite")


# <font color="aqua">Adaboost</font> Model

### Read the data

In [5]:
x_data_d, y_data_d = mlq.l(read_data)(kind='Numerical', select=False)
#x_data_d = x_data_d[['Weight', 'Height']]


### Initiate model

In [8]:
clf = mlq.l(XGBClassifier)()


# Lets do random search on a wide range of hyperparameters including regularization using random search

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint, loguniform

params = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(1, 10),
    'learning_rate': loguniform(0.001, 1),
    'gamma': uniform(0, 1),
    'min_child_weight': randint(1, 10),
    'subsample': uniform(0, 1),
    'colsample_bytree': uniform(0, 1),
    'colsample_bylevel': uniform(0, 1),
    'colsample_bynode': uniform(0, 1),
    'reg_alpha': loguniform(0.001, 1),
    'reg_lambda': loguniform(0.001, 1),
    'scale_pos_weight': uniform(0, 1),
    'base_score': uniform(0, 1),
    'booster': ['gbtree', 'gblinear', 'dart'],
    'importance_type': ['gain', 'weight', 'cover', 'total_gain', 'total_cover']
}

clf = RandomizedSearchCV(clf, params, n_iter=150, cv=5, verbose=1, n_jobs=-1)
clf.fit(x_data_d, y_data_d)


Fitting 5 folds for each of 150 candidates, totalling 750 fits
[10:45:11] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_21wtzqx5vy/croot/xgboost-split_1675457780668/work/src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[10:45:11] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_21wtzqx5vy/croot/xgboost-split_1675457780668/work/src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[10:45:11] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_21wtzqx5vy/croot/xgboost-split_1675457780668/work/src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[10:45:11] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_21wtzqx5vy/croot/xgboost-split_1675457780668/work/src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[10:45:11] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_21wtzqx5vy/croot/xgboost-split_1675457780668/work/src/learner.cc:767: 
Parameters: { "sc

In [10]:

# save hyperparameters
opt_params = clf.best_params_
save_hyperparameters(model_name, opt_params)

clf = mlq.l(XGBClassifier)(**opt_params)


### Evaluate Model Bias

In [12]:
print("Best hyperparameters:", opt_params)
# make max_depth and n_estimators integers
opt_params['max_depth'] = int(opt_params['max_depth'])
opt_params['n_estimators'] = int(opt_params['n_estimators'])
clf = mlq.l(XGBClassifier)(**opt_params)

clf.fit(x_data_d, y_data_d)
y_pred = clf.predict(x_data_d)
train_metrics = classification_report(y_data_d, y_pred, digits=3)
train_acc, train_wf1 = get_metrics(train_metrics)
print(train_metrics)

Best hyperparameters: {'base_score': 0.43468598597467867, 'booster': 'gbtree', 'colsample_bylevel': 0.8817029180590151, 'colsample_bynode': 0.8218197570762392, 'colsample_bytree': 0.7936080504044924, 'gamma': 0.043251077619481104, 'importance_type': 'weight', 'learning_rate': 0.07992860787936747, 'max_depth': 5, 'min_child_weight': 2, 'n_estimators': 149, 'reg_alpha': 0.0014354063031935301, 'reg_lambda': 0.20087250276400015, 'scale_pos_weight': 0.9928503684505409, 'subsample': 0.5025524216894568}
[11:33:31] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_21wtzqx5vy/croot/xgboost-split_1675457780668/work/src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

              precision    recall  f1-score   support

           0      1.000     0.995     0.997       190
           1      0.995     1.000     0.998       201
           2      1.000     1.000     1.000       406
           3      1.000     1.000     1.000       680

    accuracy                    

### Evaluate Model Generalization

In [11]:
kfold=cross_validation(clf, x_data_d, y_data_d, k=[10], n_repeats=[5], random_state=1,loo=False)

val_wf1 = kfold.get(f'5-Repeated 10-fold')[0]

[11:31:14] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_21wtzqx5vy/croot/xgboost-split_1675457780668/work/src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[11:31:14] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_21wtzqx5vy/croot/xgboost-split_1675457780668/work/src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[11:31:14] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_21wtzqx5vy/croot/xgboost-split_1675457780668/work/src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[11:31:15] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_21wtzqx5vy/croot/xgboost-split_1675457780668/work/src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[11:31:15] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_21wtzqx5vy/croot/xgboost-split_1675457780668/work/src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[11:31:16] WARNING: /var/folde

5-Repeated 10-fold
0.9701


#### Save Model

In [ ]:
save_model(model_name, clf)

### Tracking

In [ ]:
mlq.log_metrics( train_wf1, val_wf1)
mlq.end_quest()
mlq.show_logs(model_name, table_dest="../../", last_k=6)